# AdaBoost

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier

In [11]:
def adaboost(X_train,X_test,y_train,y_test,estimater_range): 
    scores =[]
    acc_scores = []
    mae_scores = []
    mse_scores =[]
    medae_scores = []
    r2_scores =[]
    for n_estimators in estimator_range:
        #create classifier
        clf = AdaBoostClassifier(n_estimators=n_estimators, algorithm="SAMME",)
        #fit data and predict
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        #errors
        scores.append(cross_val_score(clf, X_test, y_test, cv=5).mean())
        acc_scores.append(metrics.accuracy_score(y_test,y_pred))
        # MAE
        mae = metrics.mean_absolute_error(y_test, y_pred)
        mae_scores.append(mae)
        # MSE
        mse = metrics.mean_squared_error(y_test, y_pred)
        mse_scores.append(mse)
        # mdAE
        medae = metrics.median_absolute_error(y_test, y_pred)
        medae_scores.append(medae)
        # R2
        r2 = metrics.r2_score(y_test, y_pred)
        r2_scores.append(r2)
    return y_pred, acc_scores, scores, mae_scores, mse_scores, medae_scores, r2_scores